[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/jkanclerz/analiza-dokumentow/blob/main/16--cleaning.ipynb)

## Czyszczenie danych
* istotne z perspektywy uwydatnienia poszukiwanych informacji


## Stemming
bazując na definicji z angielskiej wikipedii jest to proces polegający na wydobyciu z wybranego wyrazu tzw. rdzenia, a więc tej jego części, która jest odporna na odmiany przez przyimki, rodzaje itp.



## Lematyzacja

pojęcie to jest bardzo podobne do powyższego, a oznacza sprowadzenie grupy wyrazów stanowiących odmianę danego zwrotu do wspólnej postaci, umożliwiającej traktowanie ich wszystkich jako te samo słowo.

## Znaki specjalne, nadmiarowe spacje, znaczniki HTML, inne jeżeli potrzeba

In [1]:
test_text = '''
Adam Mickiewicz\r\n\r\nDziady. Widowisko, część I\r\n\r\n\r\n\r\n/ Prawa strona teatru — Dziewica w samotnym pokoju — na boku ksiąg mnóstwo, fortepiano, okno z lewej strony w pole; na prawej wielkie zwierciadło; świeca gasnąca na stole i księga rozłożona (romans Valerie). /\r\n\r\n\r\nDZIEWICA\r\n\r\n/ wstaje od stołu /\r\n\r\nŚwieco niedobra! właśnie pora była zgasnąć!\r\nI nie mogłam doczytać — czyż podobna zasnąć?\r\nWaleryjo! Gustawie! anielski Gustawie!\r\nAch, tak mi często o was śniło się na jawie,\r\nA przez sen — będę z wami, Pan Bóg wie dopóki! -----\r\nTa lektura, podobnie jak tysiące innych, dostępna jest na stronie wolnelektury.pl.\r\nWersja lektury w opracowaniu merytorycznym i krytycznym (przypisy i motywy) dostępna jest na stronie http://wolnelektury.pl/katalog/lektura/dziady-dziady-widowisko-czesc-i.\r\n\r\n
'''


In [2]:
import re
def preprocess_text(text):
    text = re.sub(r"^.*\n","", text)
    text = re.sub(u"[ \n]+", " ", text) # newlines -> spaces
    text = re.sub(u"[ \r]+", " ", text) # \r -> spaces
    text = text.strip()
    text = re.sub(r"----- Ta lektura.*","", text)


    return text


assert ('\n' not in preprocess_text(test_text))
assert ('\r' not in preprocess_text(test_text))
assert ('  ' not in preprocess_text(test_text))
assert ('Ta lektura,' not in preprocess_text(test_text))


In [10]:
test_text = preprocess_text(test_text)

In [4]:
def split_to_sentences(text):
  return [re.sub(r"^ ","",l) for l in re.split('\.|,|\?|!|:', text)]

assert(['hello world', "Hello John"] == split_to_sentences("hello world! Hello John"))

### interpunkcja

In [5]:
import string 
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [6]:
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct

In [7]:
assert(remove_punct("hello, world!") == "hello world")

In [11]:
remove_punct(test_text)

'Adam Mickiewicz Dziady Widowisko część I  Prawa strona teatru — Dziewica w samotnym pokoju — na boku ksiąg mnóstwo fortepiano okno z lewej strony w pole na prawej wielkie zwierciadło świeca gasnąca na stole i księga rozłożona romans Valerie  DZIEWICA  wstaje od stołu  Świeco niedobra właśnie pora była zgasnąć I nie mogłam doczytać — czyż podobna zasnąć Waleryjo Gustawie anielski Gustawie Ach tak mi często o was śniło się na jawie A przez sen — będę z wami Pan Bóg wie dopóki '

### Stop words

In [13]:
pip install requests


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
import requests

In [16]:
stop_words = (requests
         .get('https://raw.githubusercontent.com/bieli/stopwords/master/polish.stopwords.txt')
         .text
         .split('\n'))

In [17]:

stop_words[:10]

['a', 'aby', 'ach', 'acz', 'aczkolwiek', 'aj', 'albo', 'ale', 'alez', 'ależ']

In [18]:
def remove_stop_words(text):
  return " ".join([word for word in text.split(' ') if word not in stop_words])

assert "Cześć czołem" == remove_stop_words("Cześć i czołem")

In [19]:
pip install pandas


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
import pandas as pd

BOOKS = pd.read_pickle('var/books.pkl')

In [22]:
BOOKS.head()

,author,filename,content
0,Zeromski,zeromski-oko-za-oko.txt,Stefan Żeromski\r\n\r\nOko za oko\r\n\r\nISBN ...
1,Zeromski,wszystko-i-nic.txt,Stefan Żeromski\r\n\r\nWszystko i nic\r\n\r\nI...
2,Zeromski,zeromski-rozdziobia-nas-kruki-wrony.txt,"Stefan Żeromski\r\n\r\nRozdzióbią nas kruki, w..."
3,Zeromski,silaczka.txt,Stefan Żeromski\r\n\r\nSiłaczka\r\n\r\nISBN 97...
4,Zeromski,syzyfowe-prace.txt,Stefan Żeromski\r\n\r\nSyzyfowe prace\r\n\r\nI...


In [23]:
BOOKS['content_txt'] = (BOOKS.content
                             .apply(preprocess_text)
                             .apply(lambda s: s.lower())
                             .apply(remove_stop_words)
                             .apply(split_to_sentences)
                            )

In [28]:
BOOK_LINES = BOOKS[['author', 'content_txt']].explode('content_txt')

In [25]:
BOOK_LINES = BOOK_LINES.reset_index(drop=True)

In [26]:
BOOK_LINES.head()

,author,content_txt
0,Zeromski,oko oko isbn 978-83-288-2962-6 zawiadowca stac...
1,Zeromski,połączyć towarzystwem
2,Zeromski,trzema kwadransami opuścił
3,Zeromski,otwarł drzwi wchodowe
4,Zeromski,peronu wprost owej klasy wiodące


In [ ]:
BOOK_LINES.groupby('author').count()

,content_txt
author,
Mickiewicz,3214
Orzeszkowa,73332
Prus,69622
Reymont,77587
Sienkiewicz,105748


Duża różnica w elementach danej klasy!! Może mieć wpływ na rezultaty

In [30]:
BOOK_LINES['words'] = BOOK_LINES.content_txt.apply(lambda s: len(s.split()))

In [31]:
BOOK_LINES.groupby('author')['words'].describe()


,count,mean,std,min,25%,50%,75%,max
author,,,,,,,,
Mickiewicz,17355.0,3.239700,2.383035,0.0,2.0,3.0,4.0,59.0
Orzeszkowa,73332.0,3.805092,2.749076,0.0,2.0,3.0,5.0,40.0
Prus,69622.0,3.643044,2.480359,0.0,2.0,3.0,5.0,35.0
Reymont,77586.0,3.336478,2.182561,0.0,2.0,3.0,4.0,40.0
Sienkiewicz,105749.0,3.389583,2.235611,0.0,2.0,3.0,4.0,28.0
Zeromski,17293.0,3.931128,2.749582,0.0,2.0,3.0,5.0,35.0


In [32]:
BOOK_LINES = BOOK_LINES[BOOK_LINES['words'] != 0]

In [33]:
BOOK_LINES.groupby('author')['words'].quantile(0.98)

author
Mickiewicz      9.0
Orzeszkowa     11.0
Prus           11.0
Reymont         9.0
Sienkiewicz    10.0
Zeromski       12.0
Name: words, dtype: float64

In [35]:
BOOK_LINES.groupby('author')['words'].describe()

,count,mean,std,min,25%,50%,75%,max
author,,,,,,,,
Mickiewicz,17341.0,3.242316,2.382217,1.0,2.0,3.0,4.0,59.0
Orzeszkowa,73230.0,3.810392,2.747317,1.0,2.0,3.0,5.0,40.0
Prus,69588.0,3.644824,2.479657,1.0,2.0,3.0,5.0,35.0
Reymont,77559.0,3.337640,2.182053,1.0,2.0,3.0,4.0,40.0
Sienkiewicz,105627.0,3.393498,2.233930,1.0,2.0,3.0,4.0,28.0
Zeromski,17274.0,3.935452,2.747999,1.0,2.0,3.0,5.0,35.0


In [39]:
pip install wordcloud

  Using cached wordcloud-1.8.2.2.tar.gz (220 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [27 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-13-x86_64-cpython-311
      creating build/lib.macosx-13-x86_64-cpython-311/wordcloud
      copying wordcloud/wordcloud_cli.py -> build/lib.macosx-13-x86_64-cpython-311/wordcloud
      copying wordcloud/_version.py -> build/lib.macosx-13-x86_64-cpython-311/wordcloud
      copying wordcloud/__init__.py -> build/lib.macosx-13-x86_64-cpython-311/wordcloud
      copying wordcloud/tokenization.py -> build/lib.macosx-13-x86_64-cpython-311/wordcloud
      copying wordcloud/wordcloud.py -> build/lib.macosx-13-x86_64-cpython-311/wordcloud
      copying wordcloud/color_from_image.py -> build/lib.macosx-13-x86_64-cpython-311/wordcloud
      copying

In [37]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

wordcloud = WordCloud(width = 400, height = 400, 
                      background_color = 'black', 
                      min_font_size = 5,
                      stop_words=stop_words,
                      max_words=1000,
                      collocations=False).generate("Ala ma kota a kot ma mleko")

plt.figure(figsize = (12, 12), facecolor = 'lavender')
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.tight_layout(pad = 2) 
plt.show()


ModuleNotFoundError: No module named 'wordcloud'